In [13]:
import time
import ReassignmentTools
import sys

In [16]:

def score_get_inverted_index(clusters, dataset, scores):
    num_clusters = max(clusters) + 1
    num_reads = len(dataset)
    inverted_index = []

    for i in range(0, num_clusters):
        inverted_index.append([])

    for i in range(0, num_reads):
        inverted_index[dataset[i][2]].append((dataset[i][1],scores[i]))

    return inverted_index



In [6]:
dataset_path = "../../all_250000_1.fq"
clusters_path =  "../../all_250000_1+RC.fasta.a16.t20.txt"
#classifier_path = "../strex_kraken2_250000.res"
#classifier_path = "../strex_centrifuge_250000.res"
classifier_path = "../../strex_kraken1_250000.res"

IsFasta = False
TotalReassignment = False
Zero = False

In [3]:
#def main(dataset_path, clusters_path, IsFasta, TotalReassignment, Zero, classifier_path):
start = time.time()

if TotalReassignment:
    print("Total reassignment has been chosen")
else:
    print("Partial reassignment has been chosen")

#dataset_lines = ReassignmentTools.load_dataset(dataset_path, IsFasta)

if IsFasta:
    dataset_lines = ReassignmentTools.load_dataset(dataset_path, IsFasta)
else:
    (dataset_lines, line_score) = ReassignmentTools.load_dataset(dataset_path, IsFasta)
    scores = ReassignmentTools.computeScore(line_score)
    

if IsFasta:
    print("Fasta file loaded")
else:
    print("Fastq file loaded")
    
    

Partial reassignment has been chosen
Fastq file loaded


In [15]:
for i in range(0,10):
    print(line_score[i])
    print(scores[i])

IIHIIIHHHIHFIGHFHIHEIIIIIIHIGIIGHIIIIGIIHIEEIFGIFFIFIIIIIBIFFCCID>AFFIECCIIAICFGDICICEE@GDFI8II4ICHI
37.7
IHIIIIIIHIIIIIHIGIIHIGIIIIHHFHIIGICIGFEFIHAIEIIIGIDEIIABIIDEIHIIBIFIIGDIII?IFIIIBIII@<CIIIIIIA;IAICI
38.16
IHIHHHIIIGGIGIIIIHIFIGGIFHIIGEFHIBIEDIDIHIIHEH?IGHIIIII?IB=IDBICIHIHIIEIIIDHIH>3ICIII1DADIC=IIIBID:I
37.41
IIIIHHGHHIHHGIIFHHHHIIHBHGFHHDGGHHIECIDGIIIIIEIIIGDGIGECEEDIIHDHIA8ED9EI>H>BHEFEIIDI8I?FIIIFIIIII@IC
37.45
HHIHHHIHFIIIIHGHEEIHHIEFHIIIIFEGG@IFFIDGIFEFIIHCEH6IIGFIHEIFIIGEHIFCI@FCHIHHIFHDI4EIABIHIIFIFIACH@II
37.65
HHHHIIHHIIHHIIHEHHIEIIFEIIIHGEHAIDEIIGDI?II@FIGIIE@DIHIEGICHG?IGIHIEHGDIID?AIEI@I;FEI?CIHIICIHI==II>
37.43
HIHHHIHIHIIHHGGHGIIIGIFIHDIHIHIGIIAGIIFGIFICGDIEIEIICIIHEHA;IGIGAH@DIIHID>ICIIIIAIC;II:IHIFFIII?II8I
37.7
IIIHIHHIHHIFGGIFIGHIIHIFEHFHHFIGHIFIIII@EFIIGIIIFIHIBIHAIFDICGF<CCI=I;IGIIIHICIIGFIA?III7IDFIDIAIHIE
37.75
HIHHHHHIIIIHIHIIIIIHIDIIIFHGEHIIHEFIIIFHIIFIFIIHIG?GHIIGIIHFDIIIGI@II:IIIFA@FII9H7DIIIIIIIIII>IBIII;
38.09
HHHHIGIIHGIHIFHIFIIIIHHIHHHIGIDGFIIIEBE

In [7]:
clusters_list = ReassignmentTools.load_clusters_result(clusters_path)
classifier_results = ReassignmentTools.load_classifier_result(classifier_path)

# new Structure : id , classifier result , cluster

dataset = ReassignmentTools.build_dataset(dataset_lines, clusters_list, classifier_results)

stop = time.time()

print("Files loaded and datasets created in time: ", stop - start)

if len(dataset_lines) != len(clusters_list):
    print("Error in input files!")
    exit()

  

Files loaded and datasets created in time:  32.79261302947998


In [17]:
  
# INVERTED INDEX

start = time.time()

inverted_index = score_get_inverted_index(clusters_list, dataset,scores)

stop = time.time()

print("Inverted index created in time: ", stop - start)

Inverted index created in time:  2.264085054397583


In [23]:
for i in range(0,18):
    print(inverted_index[i])

[('76759', 37.7)]
[('76759', 38.16), ('76759', 37.35)]
[('1215088', 37.41)]
[('76759', 37.45), ('76759', 37.52), ('76759', 37.8)]
[('76759', 37.65)]
[('1114970', 37.43), ('80840', 37.53)]
[('76759', 37.7), ('76759', 37.49), ('76759', 37.92)]
[('1215088', 37.75), ('1215088', 37.57), ('1215088', 37.2), ('1215088', 37.58)]
[('76759', 38.09), ('76759', 36.83), ('76759', 37.52)]
[('76759', 38.26)]
[('76759', 37.28)]
[('76759', 37.48), ('76759', 37.59), ('76759', 37.46), ('76759', 38.18), ('76759', 37.4), ('76759', 38.0)]
[('76759', 37.52)]
[('1196325', 37.51), ('1435044', 37.3)]
[('136845', 36.96), ('136845', 37.94), ('136845', 37.55)]
[('0', 37.2)]
[('76759', 37.99), ('76759', 37.55), ('76759', 38.05), ('76759', 36.75)]
[('76759', 37.93), ('136845', 38.11), ('136845', 37.66)]


In [26]:
def score_search(cluster):
    label_dict = {}

    #pair = (label,score)
    for pair in cluster:

        if pair[0] in list(label_dict):
            label_dict[pair[0]] = label_dict[pair[0]] + pair[1] 
        else:
            # if it does not exist, it's automatically created
            label_dict[pair[0]] = pair[1]

    return label_dict

In [27]:
start = time.time()


max_label_per_cluster_list = []
max_label_list = []

for cluster in inverted_index:

    # return a dictionary with {label: score} pairs that appear in the examinated cluster
    label_dict = score_search(cluster)

    ##if ZERO VERSION:
    if Zero:
        # return a pair [label, frequency], where label is the label with max frequency and frequency is max frequency
        max_label = ReassignmentTools.get_max_label_zero_version(label_dict)

    else:
        # return a pair [label, frequency], where label is the label with max frequency and frequency is max frequency
        max_label = ReassignmentTools.get_max_label(label_dict)

    # append the label with max frequncy in the list of all max labels that is used in the reassignment step
    max_label_list.append(max_label[0])

    # append the triplet [max label, max frequency, number of total reads in the cluster]
    max_label_per_cluster_list.append([max_label[0], max_label[1], len(cluster)])


if TotalReassignment:
    reassigned_classification = ReassignmentTools.total_reassignment(dataset, max_label_list)
else:
    reassigned_classification = ReassignmentTools.partial_reassignment(dataset, max_label_list)
stop = time.time()

print("Classes have been elaborated in: ", stop - start)

ReassignmentTools.printResults(dataset_path, classifier_path, TotalReassignment, reassigned_classification, Zero)

print("Done")

Classes have been elaborated in:  10.440802097320557
File  strex_kraken1_25000_SP.res  created.
Done
